In [16]:
import spacy
from spacy import displacy
import os
from graphviz import Digraph


In [29]:
class Conll_8best_Read():
    def __init__(self,dataset):
        self.dataset=dataset
    #这个函数没测试过 不过多个的没问题，这个应该也没啥问题
    def read_single_file(self,file_path,file_name):
        #path = E:\\baiduyunxiazai\\zx
        #file_path = "E:\\baiduyunxiazai\\zx\\top1\\notMatch.1"
        paths=['top1','top2','top3','top4','top5','top6','top7','top8']
        #file_path = "E:\\baiduyunxiazai\\zx\\top1\\notMatch.1"
        #file_path+paths[j]+notMatch.alldata
        file = os.path.join(file_path,paths[0],file_name)
        with open(file,'r', encoding='utf-8') as fb:
            for line in fb:
                line = line.strip('\n') 
                if line:
                    self.dataset.append(line.split('\t'))
                else:
                    self.dataset.append([])
        return self.dataset
         
    def read_mul_file(self,file_path,file_name):
        #一共973句话 算上']'
        #path = E:\\baiduyunxiazai\\zx
        paths=['top1','top2','top3','top4','top5','top6','top7','top8']
        #file_path = "E:\\baiduyunxiazai\\zx\\top1\\notMatch.1"
        for j in range(8):
            #file_path+paths[j]+notMatch.alldata
            file = os.path.join(file_path,paths[j],file_name)
            print(file)
            with open(file,'r', encoding='utf-8') as fb:
                for line in fb:
                    line = line.strip('\n') 
                    if line:
                        self.dataset.append(line.split('\t'))
                    else:
                        self.dataset.append([])
        return self.dataset
         

class Sentence(object):
    def __init__(self):
        self.sen = "" #用来判断句子是否相同
        self.idx= [] #存每一个单词的序号
        self.word = [] #存每一个单词
        self.tag = [] #存储标签
        self.dep = [] #存储依赖
        self.rel = [] #依赖关系
        self.Arc={} #字典
        self.special_name=[]
        
    def equal(self,a,b):
        if a == b:
            return True
        else:
            return False
    def dep_tree(self):
        #这里面把所有的弧都取出来
        for i in range(len(self.word)):
            if self.dep[i] == '0':
                self.Arc[self.special_name[i]] = 'root'
            else:
                for j in range(len(self.word)):
                    if self.idx[j] == self.dep[i]:
                        self.Arc[self.special_name[i]] = self.special_name[j]
                
            


In [30]:
def load_data_conll(dataset):
    Sen = Sentence()
    sente = [Sentence() for i in range(1)]
    for i in range(len(dataset)):
        if dataset[i]:
            Sen.sen = Sen.sen + dataset[i][1]
            Sen.word.append(dataset[i][1])
            Sen.idx.append(dataset[i][0])
            Sen.tag.append(dataset[i][3])
            Sen.dep.append(dataset[i][6])
            Sen.rel.append(dataset[i][7])
            Sen.special_name.append(str(dataset[i][0])+'|'+str(dataset[i][1]+'|'+str(dataset[i][3])))
        else:
            Sen.dep_tree()
            sente.append(Sen)
            Sen = Sentence()
    del(sente[0])
    print(sente[0].sen)
    return sente

def find_sent(sen):
    se = set() #存储所有已经出现了的句子
    dic = {"":[]} #为每个句子存储所有它的index
    for i in range(len(sen)):
        if sen[i].sen not in se:
            dic[sen[i].sen]=[i] #为每个句子建立一个空集合
        else:
            continue #这个句子我们就不统计了 进行一个值的跳过
        
        for j in range(i+1,len(sen)):
            if sen[j].sen not in se and sen[i].sen == sen[j].sen: 
                dic[sen[i].sen].append(j)
        se.add(sen[i].sen)
    del(dic[""])
    return se,dic

#生成graphiz类型的dependency_tree树
def generate_DPtree_graphiz(sen,st,dic):
    #sen是所有句子的集合，str是
    #我们需要为每一个句子生成一棵树，这里的话我们可以选择生成那个句子的树
    name = []
    for i in dic[st]:
        g = Digraph('依存树'+str(i))
        g.node(name='root')
        #需要让根节点指向0的位置
        for j in range(len(sen[i].word)):
            #如果需要添加词的属性需要加上下边这一行，否则就不需要
            #name.append(str(sen[i].word[j])+'|'+str(sen[i].tag[j]))
            g.node(sen[i].special_name[j],fontname="Microsoft YaHei")
        for j in range(len(sen[i].word)):
            if sen[i].rel[j]=='root':
                g.edge('root',sen[i].special_name[j],label=str(sen[i].dep[j]))
                
            else:
                g.edge(sen[i].Arc[sen[i].special_name[j]],sen[i].special_name[j],label=str(sen[i].rel[j]))
        g.view()

In [31]:
file_path="E:/baiduyunxiazai/zx/"
fila_name="notMatch.alldata"
dataset=[]
data = Conll_8best_Read(dataset)
dataset = data.read_mul_file(file_path,fila_name)
sen = load_data_conll(dataset)
se,dic = find_sent(sen)
generate_DPtree_graphiz(sen,"只不过，在这之后，张小凡也感觉到，从绑在自己右手臂膀上的那个奇异法宝，却似乎散发着与烧火棍相反的，带着一丝温暖的气息，传进自己的身体。",dic)

E:/baiduyunxiazai/zx/top1\notMatch.alldata
E:/baiduyunxiazai/zx/top2\notMatch.alldata
E:/baiduyunxiazai/zx/top3\notMatch.alldata
E:/baiduyunxiazai/zx/top4\notMatch.alldata
E:/baiduyunxiazai/zx/top5\notMatch.alldata
E:/baiduyunxiazai/zx/top6\notMatch.alldata
E:/baiduyunxiazai/zx/top7\notMatch.alldata
E:/baiduyunxiazai/zx/top8\notMatch.alldata
」


In [20]:
print(dic["只不过，在这之后，张小凡也感觉到，从绑在自己右手臂膀上的那个奇异法宝，却似乎散发着与烧火棍相反的，带着一丝温暖的气息，传进自己的身体。"])
print(sen[161].sen)

[161, 562, 963, 1364, 1765, 2166, 2567, 2968]
只不过，在这之后，张小凡也感觉到，从绑在自己右手臂膀上的那个奇异法宝，却似乎散发着与烧火棍相反的，带着一丝温暖的气息，传进自己的身体。


In [26]:
print(sen[161].Arc['，'])
print(sen[161].word)

传进
['只', '不过', '，', '在', '这', '之后', '，', '张小凡', '也', '感觉', '到', '，', '从', '绑', '在', '自己', '右', '手', '臂膀', '上', '的', '那', '个', '奇异', '法宝', '，', '却', '似乎', '散发', '着', '与', '烧火棍', '相反', '的', '，', '带', '着', '一', '丝', '温暖', '的', '气息', '，', '传进', '自己', '的', '身体', '。']
